## 랩 2: 메모리 추가로 에이전트 개인화

### 개요

랩 1에서는 로컬 세션에서 단일 사용자에게 잘 작동하는 고객 지원 에이전트를 구축했습니다. 그러나 실제 고객 지원은 로컬 환경에서 실행되는 단일 사용자를 넘어서 확장되어야 합니다.

**프로덕션에서 에이전트**를 실행할 때 필요한 것들:
- **다중 사용자 지원**: 수천 명의 고객을 동시에 처리
- **지속적 저장소**: 세션 수명 주기를 넘어서 대화 저장
- **장기 학습**: 고객 선호도와 행동 패턴 추출
- **세션 간 연속성**: 다른 상호작용에서도 고객 기억

**워크샵 진행 상황:**
- **랩 1 (완료)**: 에이전트 프로토타입 생성 - 기능적인 고객 지원 에이전트 구축
- **랩 2 (현재)**: 메모리로 향상 - 대화 컨텍스트와 개인화 추가
- **랩 3**: 게이트웨이 및 아이덴티티로 확장 - 에이전트 간 도구 안전하게 공유
- **랩 4**: 프로덕션 배포 - 관찰 가능성을 갖춘 AgentCore 런타임 사용
- **랩 5**: 사용자 인터페이스 구축 - 고객 대면 애플리케이션 생성


이 랩에서는 금붕어 에이전트(몇 초 만에 대화를 잊어버리는)를 스마트한 개인화 어시스턴트로 변환시키는 누락된 지속성과 학습 레이어를 추가하겠습니다.

메모리는 지능의 중요한 구성 요소입니다. 대형 언어 모델(LLM)은 인상적인 기능을 가지고 있지만, 대화 간 지속적인 메모리가 부족합니다. [Amazon Bedrock AgentCore 메모리](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/메모리-getting-started.html)는 AI 에이전트가 시간이 지나도 컨텍스트를 유지하고, 중요한 사실을 기억하며, 일관되고 개인화된 경험을 제공할 수 있도록 하는 관리형 서비스를 제공하여 이러한 한계를 해결합니다.

AgentCore Memory는 두 가지 수준에서 작동합니다:
- **단기 메모리**: 단일 상호작용 또는 밀접하게 관련된 세션 내에서 연속성을 제공하는 즉시 대화 컨텍스트와 세션 기반 정보
- **장기 메모리**: 시간이 지나면서 개인화된 경험을 가능하게 하는 사실, 선호도, 요약을 포함하여 여러 대화에 걸쳐 추출되고 저장되는 지속적 정보

### 랩 2 아키텍처
<div style="text-align:left">
    <img src="images/architecture_lab2_memory.png" width="75%"/>
</div>

*지속적인 단기 및 장기 메모리 기능을 갖춘 다중 사용자 에이전트*

### 전제조건

* 적절한 권한을 가진 **AWS 계정**
* 로컬에 설치된 **Python 3.10+**
* 자격 증명으로 구성된 **AWS CLI**
* [Amazon Bedrock](https://docs.aws.amazon.com/bedrock/latest/userguide/model-access.html)에서 활성화된 **Anthropic Claude 3.7**
* 다음 셀에서 설치될 **Strands Agents** 및 기타 라이브러리

### 1단계: 라이브러리 가져오기

AgentCore Memory를 위한 라이브러리를 가져오겠습니다. 이를 위해 AgentCore 기능을 사용하는 데 도움이 되는 경량 래퍼인 [Amazon Bedrock AgentCore Python SDK](https://github.com/aws/bedrock-agentcore-sdk-python)를 사용하겠습니다.

In [ ]:
import logging

# Import agentCore Memory
from bedrock_agentcore.memory import MemoryClient
from bedrock_agentcore.memory.constants import StrategyType

from strands.hooks import AfterInvocationEvent, HookProvider, HookRegistry, MessageAddedEvent

import boto3
from boto3.session import Session

boto_session = Session()
REGION = boto_session.region_name

logger = logging.getLogger(__name__)

from lab_helpers.utils import get_ssm_parameter, put_ssm_parameter

### 2단계: Bedrock AgentCore 메모리 리소스 생성

Amazon Bedrock AgentCore Memory는 AI 에이전트에 지속적인 메모리 기능을 제공하는 완전 관리형 서비스입니다.

#### AgentCore 메모리 개념:

1. **단기 메모리 (STM)**: 세션 내에서 대화 컨텍스트를 즉시 저장
2. **장기 메모리 (LTM)**: STM을 비동기적으로 처리하여 의미 있는 패턴, 선호도, 사실 추출
3. **메모리 전략**: 정보 추출 및 조직화를 위한 다양한 접근 방식:
   - **USER_PREFERENCE**: 고객 선호도, 행동, 패턴 학습
   - **SEMANTIC**: 유사성 검색을 위해 벡터 임베딩을 사용하여 사실 정보 저장
4. **네임스페이스**: 고객 및 컨텍스트 유형별 메모리의 논리적 그룹화. 다음 두 네임스페이스를 생성합니다:
- `support/customer/{actorId}/preferences`: 고객 선호도 및 행동 패턴
- `support/customer/{actorId}/semantic`: 사실 정보 및 대화 기록

이 구조는 각 고객의 정보가 분리되고 쉽게 검색할 수 있는 다중 테넌트 메모리를 가능하게 합니다.

#### 메모리 생성 프로세스:

메모리 리소스 생성에는 기본 인프라(벡터 데이터베이스, 처리 파이프라인 등) 프로비저닝이 포함됩니다. AWS가 백그라운드에서 관리형 서비스를 설정하는 동안 일반적으로 2-3분이 소요됩니다.

In [ ]:
memory_client = MemoryClient(region_name=REGION)
memory_name = "CustomerSupportMemory"

def create_or_get_memory_resource():
    try:
        memory_id = get_ssm_parameter("/app/customersupport/agentcore/memory_id")
        memory_client.gmcp_client.get_memory(memoryId=memory_id)
        return memory_id
    except:
        try:
            strategies = [
                {
                    StrategyType.USER_PREFERENCE.value: {
                        "name": "CustomerPreferences",
                        "description": "Captures customer preferences and behavior",
                        "namespaces": ["support/customer/{actorId}/preferences"],
                    }
                },
                {
                    StrategyType.SEMANTIC.value: {
                        "name": "CustomerSupportSemantic",
                        "description": "Stores facts from conversations",
                        "namespaces": ["support/customer/{actorId}/semantic"],
                    }
                },
            ]
            print("Creating AgentCore Memory resources. This will take 2-3 minutes...")
            print("While we wait, let's understand what's happening behind the scenes:")
            print("• Setting up managed vector databases for semantic search")
            print("• Configuring memory extraction pipelines")
            print("• Provisioning secure, multi-tenant storage")
            print("• Establishing namespace isolation for customer data")
            # *** AGENTCORE MEMORY USAGE *** - Create memory resource with semantic strategy
            response = memory_client.create_memory_and_wait(
                name=memory_name,
                description="Customer support agent memory",
                strategies=strategies,
                event_expiry_days=90,          # Memories expire after 90 days
            )
            memory_id = response["id"]
            try:
                put_ssm_parameter("/app/customersupport/agentcore/memory_id", memory_id)
            except:
                raise
            return memory_id
        except Exception as e:
            print(f"Failed to create memory resource: {e}")
            return None

In [ ]:
memory_id = create_or_get_memory_resource()
if memory_id:
    print("✅ AgentCore Memory created successfully!")
    print(f"Memory ID: {memory_id}")
else:
    print("Memory resource not created. Try Again !")

## 3단계: 이전 고객 상호작용 시드 데이터 생성

**왜 메모리에 시드 데이터를 생성하나요?**

프로덕션 환경에서는 에이전트가 고객 상호작용을 통해 자연스럽게 메모리를 축적합니다. 하지만 이 랩에서는 실제 대화를 기다리지 않고 장기 메모리(LTM)가 어떻게 작동하는지 보여주기 위해 과거 대화를 시드 데이터로 생성합니다.

**메모리 처리 작동 방식:**
1. `create_event`는 상호작용을 **단기 메모리**(STM)에 즉시 저장
2. STM은 **장기 메모리** 전략에 의해 비동기적으로 처리
3. LTM은 향후 검색을 위해 패턴, 선호도, 사실을 추출

이를 실제로 확인하기 위해 일부 고객 기록을 시드 데이터로 생성해보겠습니다:

In [ ]:
# List existing memory resources
for memory in memory_client.list_memories():
    print(f"Memory Arn: {memory.get('arn')}")
    print(f"Memory ID: {memory.get('id')}")
    print("--------------------------------------------------------------------")

# Seed with previous customer interactions
CUSTOMER_ID = "customer_001"

previous_interactions = [
    ("I'm having issues with my MacBook Pro overheating during video editing.","USER"),
    ("I can help with that thermal issue. For video editing workloads, let's check your Activity Monitor and adjust performance settings. Your MacBook Pro order #MB-78432 is still under warranty.", "ASSISTANT"),
    ("What's the return policy on gaming headphones? I need low latency for competitive FPS games", "USER"),
    ("For gaming headphones, you have 30 days to return. Since you're into competitive FPS, I'd recommend checking the audio latency specs - most gaming models have <40ms latency.", "ASSISTANT"),
    ("I need a laptop under $1200 for programming. Prefer 16GB RAM minimum and good Linux compatibility. I like ThinkPad models.", "USER"),
    ("Perfect! For development work, I'd suggest looking at our ThinkPad E series or Dell XPS models. Both have excellent Linux support and 16GB RAM options within your budget.", "ASSISTANT"),
]

# Save previous interactions
if memory_id:
    try:
        memory_client.create_event(
            memory_id=memory_id,
            actor_id=CUSTOMER_ID,
            session_id="previous_session",
            messages=previous_interactions
        )
        print("✅ Seeded customer history successfully")
        print("📝 Interactions saved to Short-Term Memory")
        print("⏳ Long-Term Memory processing will begin automatically...")
    except Exception as e:
        print(f"⚠️ Error seeding history: {e}")

### 메모리 처리 이해

`create_event`로 이벤트를 생성한 후, AgentCore Memory는 데이터를 두 단계로 처리합니다:

1. **즉시**: 메시지가 단기 메모리(STM)에 저장
2. **비동기**: STM이 장기 메모리(LTM) 전략으로 처리

LTM 처리는 시스템이 다음 작업을 수행하므로 일반적으로 20-30초가 소요됩니다:
- 대화 패턴 분석
- 고객 선호도 및 행동 추출
- 사실 정보에 대한 의미적 임베딩 생성
- 효율적인 검색을 위해 네임스페이스별로 메모리 구성

고객 선호도를 검색하여 장기 메모리 처리가 완료되었는지 확인해보겠습니다:

In [ ]:
import time

# Wait for Long-Term Memory processing to complete
print("🔍 Checking for processed Long-Term Memories...")
retries = 0
max_retries = 6  # 1 minute wait

while retries < max_retries:
    memories = memory_client.retrieve_memories(
        memory_id=memory_id,
        namespace=f"support/customer/{CUSTOMER_ID}/preferences",
        query="can you summarize the support issue"
    )
    
    if memories:
        print(f"✅ Found {len(memories)} preference memories after {retries * 10} seconds!")
        break
    
    retries += 1
    if retries < max_retries:
        print(f"⏳ Still processing... waiting 10 more seconds (attempt {retries}/{max_retries})")
        time.sleep(10)
    else:
        print("⚠️ Memory processing is taking longer than expected. This can happen with overloading..")
        break

print("🎯 AgentCore Memory automatically extracted these customer preferences from our seeded conversations:")
print("=" * 80)

for i, memory in enumerate(memories, 1):
    if isinstance(memory, dict):
        content = memory.get('content', {})
        if isinstance(content, dict):
            text = content.get('text', '')
            print(f"  {i}. {text}")

### 의미적 메모리 탐색

의미적 메모리는 벡터 임베딩을 사용하여 대화에서 사실 정보를 저장합니다. 이를 통해 관련 사실과 컨텍스트의 유사성 기반 검색이 가능합니다.

In [ ]:
import time
# Retrieve semantic memories (factual information)
while True:
    semantic_memories = memory_client.retrieve_memories(
        memory_id=memory_id,
        namespace=f"support/customer/{CUSTOMER_ID}/semantic",
        query="information on the technical support issue"
    )
    print("🧠 AgentCore Memory identified these factual details from conversations:")
    print("=" * 80)
    if memories:
        break
    time.sleep(10)
for i, memory in enumerate(semantic_memories, 1):
    if isinstance(memory, dict):
        content = memory.get('content', {})
        if isinstance(content, dict):
            text = content.get('text', '')
            print(f"  {i}. {text}")

## 3단계: 에이전트 상호작용을 저장하고 검색하기 위한 Strands 훅 구현

이제 Strands의 훅 시스템을 사용하여 AgentCore Memory를 에이전트와 통합하겠습니다. 이는 모든 에이전트 대화와 원활하게 작동하는 자동 메모리 레이어를 생성합니다.

- **MessageAddedEvent**: 메시지가 대화에 추가될 때 트리거되어 고객 컨텍스트를 검색하고 주입할 수 있게 합니다
- **AfterInvocationEvent**: 에이전트 응답 후 실행되어 상호작용을 메모리에 자동으로 저장할 수 있게 합니다

훅 시스템은 수동 개입 없이 메모리 작업이 자동으로 수행되도록 보장하여 고객 컨텍스트가 대화 전반에 걸쳐 보존되는 원활한 경험을 만듭니다.

훅을 생성하기 위해 `HookProvider` 클래스를 확장하겠습니다:


In [ ]:
class CustomerSupportMemoryHooks(HookProvider):
    """Memory hooks for customer support agent"""

    def __init__(
        self, memory_id: str, client: MemoryClient, actor_id: str, session_id: str
    ):
        self.memory_id = memory_id
        self.client = client
        self.actor_id = actor_id
        self.session_id = session_id
        self.namespaces = {
            i["type"]: i["namespaces"][0]
            for i in self.client.get_memory_strategies(self.memory_id)
        }

    def retrieve_customer_context(self, event: MessageAddedEvent):
        """Retrieve customer context before processing support query"""
        messages = event.agent.messages
        if (
            messages[-1]["role"] == "user"
            and "toolResult" not in messages[-1]["content"][0]
        ):
            user_query = messages[-1]["content"][0]["text"]

            try:
                all_context = []

                for context_type, namespace in self.namespaces.items():
                    # *** AGENTCORE MEMORY USAGE *** - Retrieve customer context from each namespace
                    memories = self.client.retrieve_memories(
                        memory_id=self.memory_id,
                        namespace=namespace.format(actorId=self.actor_id),
                        query=user_query,
                        top_k=3,
                    )
                    # Post-processing: Format memories into context strings
                    for memory in memories:
                        if isinstance(memory, dict):
                            content = memory.get("content", {})
                            if isinstance(content, dict):
                                text = content.get("text", "").strip()
                                if text:
                                    all_context.append(
                                        f"[{context_type.upper()}] {text}"
                                    )

                # Inject customer context into the query
                if all_context:
                    context_text = "\n".join(all_context)
                    original_text = messages[-1]["content"][0]["text"]
                    messages[-1]["content"][0][
                        "text"
                    ] = f"Customer Context:\n{context_text}\n\n{original_text}"
                    logger.info(f"Retrieved {len(all_context)} customer context items")

            except Exception as e:
                logger.error(f"Failed to retrieve customer context: {e}")

    def save_support_interaction(self, event: AfterInvocationEvent):
        """Save customer support interaction after agent response"""
        try:
            messages = event.agent.messages
            if len(messages) >= 2 and messages[-1]["role"] == "assistant":
                # Get last customer query and agent response
                customer_query = None
                agent_response = None

                for msg in reversed(messages):
                    if msg["role"] == "assistant" and not agent_response:
                        agent_response = msg["content"][0]["text"]
                    elif (
                        msg["role"] == "user"
                        and not customer_query
                        and "toolResult" not in msg["content"][0]
                    ):
                        customer_query = msg["content"][0]["text"]
                        break

                if customer_query and agent_response:
                    # *** AGENTCORE MEMORY USAGE *** - Save the support interaction
                    self.client.create_event(
                        memory_id=self.memory_id,
                        actor_id=self.actor_id,
                        session_id=self.session_id,
                        messages=[
                            (customer_query, "USER"),
                            (agent_response, "ASSISTANT"),
                        ],
                    )
                    logger.info("Saved support interaction to memory")

        except Exception as e:
            logger.error(f"Failed to save support interaction: {e}")

    def register_hooks(self, registry: HookRegistry) -> None:
        """Register customer support memory hooks"""
        registry.add_callback(MessageAddedEvent, self.retrieve_customer_context)
        registry.add_callback(AfterInvocationEvent, self.save_support_interaction)
        logger.info("Customer support memory hooks registered")



## 4단계: 메모리를 갖춘 고객 지원 에이전트 생성

다음으로, 랩 1에서와 같이 고객 지원 에이전트를 구현하지만, 이번에는 `CustomerSupportMemoryHooks` 클래스를 인스턴스화하고 메모리 훅을 에이전트 생성자에 전달합니다.

In [ ]:
import uuid

from strands import Agent
from strands.models import BedrockModel

from lab_helpers.lab1_strands_agent import (
    SYSTEM_PROMPT,
    get_return_policy, web_search,
    get_product_info, MODEL_ID
)

SESSION_ID = str(uuid.uuid4())
memory_hooks = CustomerSupportMemoryHooks(memory_id, memory_client, CUSTOMER_ID, SESSION_ID)


# Initialize the Bedrock model (Anthropic Claude 3.7 Sonnet)
model = BedrockModel(
    model_id=MODEL_ID,
    region_name=REGION
)

# Create the customer support agent with all 5 tools
agent = Agent(
    model=model,
    hooks=[memory_hooks], # Pass Memory Hooks
    tools=[
        get_product_info,      # Tool 1: Simple product information lookup
        get_return_policy,      # Tool 2: Simple return policy lookup
        web_search,
    ],
    system_prompt=SYSTEM_PROMPT
)

## 5단계: 개인화된 에이전트 테스트

메모리가 향상된 에이전트를 테스트해보겠습니다! 고객의 과거 선호도를 사용하여 개인화된 추천을 제공하는 방법을 확인해보세요.

에이전트는 자동으로:
1. 메모리에서 관련 고객 컨텍스트를 검색
2. 해당 컨텍스트를 사용하여 응답을 개인화
3. 향후 사용을 위해 이 새로운 상호작용을 저장

In [ ]:
from IPython.display import display, Markdown

print("🎧 Testing headphone recommendation with customer memory...\n\n")
response1 = agent("Which headphones would you recommend?")

In [ ]:
print("\n💻 Testing laptop preference recall...\n\n")
response2 = agent("What is my preferred laptop brand and requirements?")

에이전트가 다음을 기억하는 방법을 주목하세요:
• 게임 선호도 (저지연 헤드폰)
• 노트북 선호도 (ThinkPad, 16GB RAM, Linux 호환성)
• 예산 제약 (노트북 $1200)
• 이전 기술적 문제 (MacBook 과열)

이것이 AgentCore Memory의 힘입니다 - 지속적이고 개인화된 고객 경험!

## 축하합니다! 🎉

**랩 2: 고객 지원 에이전트에 메모리 추가**를 성공적으로 완료했습니다!

### 달성한 내용:

- Amazon Bedrock AgentCore Memory로 서버리스 관리형 메모리 생성
- 사용자 선호도 및 의미적(사실) 정보를 저장하는 장기 메모리 구현
- Strands Agents에서 제공하는 훅 메커니즘을 사용하여 AgentCore Memory를 고객 지원 에이전트와 통합

##### 다음 단계 [랩 3 - 게이트웨이 및 아이덴티티로 확장 →](lab-03-agentcore-게이트웨이.ipynb)

## 리소스
- [Amazon Bedrock 에이전트 Core 메모리](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/메모리.html)
- [Amazon Bedrock AgentCore 메모리 Deep Dive blog](https://aws.amazon.com/blogs/machine-learning/amazon-bedrock-agentcore-메모리-building-context-aware-agents/)
- [Strands Agents Hooks Documentation](https://strandsagents.com/latest/documentation/docs/user-guide/concepts/agents/hooks/?h=hooks)